<a href="https://colab.research.google.com/github/Niel-77/Geminiagent/blob/main/Two_Agent_team_using_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Creating two agent team using Gemini
This is the notebook for creating conversable agent using gemini api. In this notebook we will create a conversable agent to solve a simple programming problem.


## Creating Gemini API key
We	can obtain Gemini API key from https://aistudio.google.com/. Gemini 1.0, Gemini 1.5 Pro and Gemini 1.5 Flash can be used for free of charge but with limits set to each of them for Requests per minutes (RPM), tokens per minute (TPM) and requests per day (RPD).

## Install Autogen with gemini features

In [ ]:
pip install pyautogen[gemini]

## Install the necessary dependencies

In [ ]:

# Install pyautogen with the required extras
!pip install pyautogen[gemini,retrievechat,lmm] --use-deprecated=legacy-resolver


## Importing the necessary functions
(All the imported functions might not have been used)

In [ ]:
import os
from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union

import chromadb
from PIL import Image
from termcolor import colored

import autogen
from autogen import Agent, AssistantAgent, ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.img_utils import _to_pil, get_image_data
from autogen.agentchat.contrib.multimodal_conversable_agent import MultimodalConversableAgent
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen.code_utils import DEFAULT_MODEL, UNKNOWN, content_str, execute_code, extract_code, infer_lang

## Creating configuration for gemini and setting API endpoint
The similar configuration model can be used for all LLM models. For using gpt along with gemini, for example, we can add another list of called *config_list_gpt* with the model name and api key. This might not be the best way to create configuration when using multiple agent team but this will help us to avoid OAI_CONFIG_LIST which I found to be a little bit confusing to use with colab.

In [ ]:
from google.colab import userdata
geminiapi=userdata.get('GEMINIAPI')
config_list_gemini = [
     {
        "model": "gemini-pro",
        "api_key": "your_gemini_api",
        "api_type": "google"
    }
]


In [ ]:
gemini_config = {
    "seed": 25,  # change the seed for different trials
    "temperature": 0,
    "config_list": config_list_gemini,
    "request_timeout": 600,
    "retry_wait_time": 120,
}

## Creating assistant called coder
Here we create a assistant called coder with system message telling that they are a professional python coder who specialize at creating error free python codes.

In [ ]:
assistant = AssistantAgent(
    name="coder",
    system_message="""
    You are a professional python coder .\
    You are a specialist in creating error free codes.\
    You write code such that even an amateur can understand it easily.\
    You say 'DONE' when you are done.""",
    llm_config={"config_list": config_list_gemini, "seed": seed}, max_consecutive_auto_reply=3
)

user_proxy = UserProxyAgent(
    "user_proxy",
    code_execution_config={"work_dir": "coding", "use_docker": False},
    human_input_mode="NEVER",
    is_termination_msg=lambda x: content_str(x.get("content")).find("TERMINATE") >= 0,
)

result = user_proxy.initiate_chat(assistant, message="Write a program to print prime numbers upto 2500.")

user_proxy (to coder):

Write a program to print prime numbers upto 2500.

--------------------------------------------------------------------------------
coder (to user_proxy):

```python
# This program prints prime numbers upto 2500

# Create a list of all numbers from 2 to 2500
numbers = list(range(2, 2501))

# Iterate over the list and remove all non-prime numbers
for i in range(2, 2501):
    if i * i > 2500:
        break
    if numbers[i - 2] == 0:
        continue
    for j in range(i * i, 2501, i):
        numbers[j - 2] = 0

# Print the list of prime numbers
print("Prime numbers upto 2500:")
for i in range(2, 2501):
    if numbers[i - 2] != 0:
        print(numbers[i - 2])

# DONE
```

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
user_proxy (to coder):

exitcode: 0 (execution succeeded)
Code output: 
Prime numbers upto 2500:
2
3
5
7
11
13
17
19
23
29
31
37
41
43
47
53
59
61
6